In [ ]:
# [1] Setup Environment & Clone Repo
import os
if os.path.exists('/kaggle/working'):
    os.chdir('/kaggle/working')
    !rm -rf ai_project

!git clone https://github.com/yangzeha/ai_project.git
%cd ai_project
!pip install -r TSB_CL_Project/requirements.txt
!apt-get update && apt-get install -y libsparsehash-dev

# [2] Compile MSBE (C++)
!g++ -O3 Similar-Biclique-Idx-main/main.cpp -o Similar-Biclique-Idx-main/msbe
!chmod +x Similar-Biclique-Idx-main/msbe

# [3] Create Data Preparation Script
import os
code = """
import os
import urllib.request
import shutil

def download_yelp2018():
    # 1. Setup Paths
    base_dir = os.path.dirname(os.path.abspath(__file__))
    dataset_dir = os.path.join(base_dir, "datasets")
    if not os.path.exists(dataset_dir):
        os.makedirs(dataset_dir)
        
    output_path = os.path.join(dataset_dir, "yelp2018.txt")
    
    # URLs from kuandeng/LightGCN repository
    base_url = "https://raw.githubusercontent.com/kuandeng/LightGCN/master/Data/yelp2018"
    files = ["train.txt", "test.txt"]
    
    print("Downloading Yelp2018 dataset...")
    
    all_interactions = []
    
    for filename in files:
        url = f"{base_url}/{filename}"
        print(f"Downloading {filename} from {url}...")
        try:
            with urllib.request.urlopen(url) as response:
                content = response.read().decode('utf-8')
                
                # Parse LightGCN format: UserID ItemID1 ItemID2 ...
                for line in content.strip().split('\\n'):
                    parts = list(map(int, line.strip().split()))
                    if len(parts) < 2: continue
                    user = parts[0]
                    items = parts[1:]
                    for item in items:
                        all_interactions.append((user, item))
                        
        except Exception as e:
            print(f"Failed to download {filename}: {e}")
            return

    if not all_interactions:
        print("Error: No data downloaded.")
        return

    # Remap IDs to ensure continuity (0...N-1)
    print("Processing data...")
    users = sorted(list(set(x[0] for x in all_interactions)))
    items = sorted(list(set(x[1] for x in all_interactions)))
    
    user_map = {u: i for i, u in enumerate(users)}
    item_map = {i: i for i, i in enumerate(items)} # Items usually start from 0 in LightGCN data but let's be safe
    
    num_users = len(users)
    num_items = len(items)
    num_edges = len(all_interactions)
    
    print(f"Stats: Users={num_users}, Items={num_items}, Interactions={num_edges}")
    
    # Save to txt format for data_utils.py
    print(f"Saving to {output_path}...")
    with open(output_path, 'w') as f:
        # Header
        f.write(f"{num_users} {num_items} {num_edges}\\n")
        # Data
        for u, i in all_interactions:
            f.write(f"{user_map[u]} {item_map[i]}\\n")
            
    print("Done! Yelp2018 dataset is ready.")
    print(f"Path: {output_path}")

if __name__ == "__main__":
    download_yelp2018()
"""
os.makedirs('TSB_CL_Project', exist_ok=True)
with open('TSB_CL_Project/prepare_yelp2018.py', 'w', encoding='utf-8') as f:
    f.write(code)

# [4] Download & Process Yelp2018
!python TSB_CL_Project/prepare_yelp2018.py

# [5] Create Quick Proof Script
import os
code = """
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import torch
import torch.optim as optim
import numpy as np
import random
import time
import matplotlib.pyplot as plt
from model import TSB_CL
from data_utils import DataUtils

# --- Configuration ---
LR = 0.001
BATCH_SIZE = 2048
EPOCHS = 50
EMBEDDING_DIM = 64
TAU = 2
EPSILON = 0.1

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

set_seed(2024)

def run_quick_proof():
    print("=== Running Quick Proof on Yelp2018 (Full Data) ===")
    
    # 1. Setup Paths
    CURRENT_DIR = os.path.dirname(os.path.abspath(__file__))
    PROJECT_ROOT = os.path.dirname(CURRENT_DIR)
    
    # Create model_path directory
    MODEL_DIR = os.path.join(CURRENT_DIR, "model_path")
    os.makedirs(MODEL_DIR, exist_ok=True)
    
    # Path to the newly prepared Yelp2018 dataset
    DATA_PATH = os.path.join(CURRENT_DIR, "datasets", "yelp2018.txt")
    
    # Path to MSBE executable
    exe_name = "msbe.exe" if os.name == 'nt' else "msbe"
    MSBE_EXE = os.path.join(PROJECT_ROOT, "Similar-Biclique-Idx-main", exe_name)

    if not os.path.exists(DATA_PATH):
        print(f"Error: Dataset not found at {DATA_PATH}")
        print("Please run 'prepare_yelp2018.py' first.")
        return

    if not os.path.exists(MSBE_EXE):
        print(f"Warning: MSBE executable not found at {MSBE_EXE}")
        print("Biclique mining might fail if not cached.")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")
    print(f"Data Path: {DATA_PATH}")

    # 2. Load Data
    utils = DataUtils(DATA_PATH, MSBE_EXE)
    all_data = utils.load_data()
    
    # 3. Split Data
    # Use Full Data (No Sampling)
    print("Using Full Dataset...")
    # random.shuffle(all_data) # Shuffle is good, but let's keep it consistent
    
    split_idx = int(len(all_data) * 0.8)
    train_data = all_data[:split_idx]
    test_data = all_data[split_idx:]
    
    print(f"Train size: {len(train_data)}, Test size: {len(test_data)}")
    
    # 4. Prepare Graph Structures
    print("Building Adjacency Matrix...")
    adj_matrix = utils.build_adj_matrix(train_data).to(device)
    
    print("Mining Bicliques from Training Data...")
    biclique_file = utils.run_msbe_mining(train_data, "yelp_train_full", tau=TAU, epsilon=EPSILON)
    H_v, H_u = utils.parse_bicliques(biclique_file)
    H_v = H_v.to(device)
    H_u = H_u.to(device)
    
    print(f"Bicliques loaded. H_u shape: {H_u.shape}, H_v shape: {H_v.shape}")
    
    # 5. Define Models
    print("Initializing Models...")
    # 1. TSB-CL Model (Biclique + Contrastive Learning)
    model_tsb = TSB_CL(utils.num_users, utils.num_items, EMBEDDING_DIM).to(device)
    
    # 2. TSB Model (Biclique ONLY, No Contrastive Learning)
    model_tsb_no_cl = TSB_CL(utils.num_users, utils.num_items, EMBEDDING_DIM).to(device)
    
    # 3. Baseline (LightGCN equivalent, No Biclique, No CL)
    model_base = TSB_CL(utils.num_users, utils.num_items, EMBEDDING_DIM).to(device) 
    
    # 4. Baseline + CL (LightGCN + Contrastive Learning)
    # We will simulate CL for LightGCN by adding noise to embeddings to create a second view
    model_base_cl = TSB_CL(utils.num_users, utils.num_items, EMBEDDING_DIM).to(device)

    opt_tsb = optim.Adam(model_tsb.parameters(), lr=LR)
    opt_tsb_no_cl = optim.Adam(model_tsb_no_cl.parameters(), lr=LR)
    opt_base = optim.Adam(model_base.parameters(), lr=LR)
    opt_base_cl = optim.Adam(model_base_cl.parameters(), lr=LR)
    
    # 6. Evaluation Function
    test_users = list(set([x[0] for x in test_data]))
    # Evaluate on 1000 users for speed during training, full eval at end if needed
    if len(test_users) > 1000:
        eval_users_subset = random.sample(test_users, 1000)
    else:
        eval_users_subset = test_users
        
    test_user_ground_truth = {}
    for u, i, _ in test_data:
        if u not in test_user_ground_truth: test_user_ground_truth[u] = set()
        test_user_ground_truth[u].add(i)
        
    train_user_items = {}
    for u, i, _ in train_data:
        if u not in train_user_items: train_user_items[u] = set()
        train_user_items[u].add(i)

    def evaluate(model, use_biclique, target_users=None):
        if target_users is None:
            target_users = eval_users_subset
            
        model.eval()
        with torch.no_grad():
            if use_biclique:
                u_g, u_l, u_final, i_emb = model(adj_matrix, (H_v, H_u))
            else:
                dummy_Hv = torch.sparse_coo_tensor(size=(1, utils.num_items)).to(device)
                dummy_Hu = torch.sparse_coo_tensor(size=(utils.num_users, 1)).to(device)
                u_g, u_l, u_final, i_emb = model(adj_matrix, (dummy_Hv, dummy_Hu))
            
            # Use u_final for prediction
            u_emb = u_final
            
            hits = 0
            ndcgs = 0
            
            for u in target_users:
                if u not in test_user_ground_truth: continue
                gt = test_user_ground_truth[u]
                
                scores = torch.matmul(u_emb[u], i_emb.t())
                
                # Mask training items
                if u in train_user_items:
                    mask_indices = list(train_user_items[u])
                    scores[mask_indices] = -float('inf')
                
                _, indices = torch.topk(scores, 20)
                preds = indices.cpu().numpy()
                
                # Recall
                hit = len(set(preds) & gt)
                hits += hit / len(gt)
                
                # NDCG
                dcg = 0
                idcg = 0
                for i, item in enumerate(preds):
                    if item in gt:
                        dcg += 1.0 / np.log2(i + 2)
                for i in range(min(len(gt), 20)):
                    idcg += 1.0 / np.log2(i + 2)
                ndcgs += dcg / idcg if idcg > 0 else 0
                
            return hits / len(target_users), ndcgs / len(target_users)

    # 7. Training Loop
    print(f"Starting Training for {EPOCHS} epochs...")
    
    users_np = np.array([x[0] for x in train_data])
    items_np = np.array([x[1] for x in train_data])
    num_batches = len(train_data) // BATCH_SIZE
    
    # History for plotting
    history = {
        'epoch': [],
        'recall_tsb': [],
        'recall_tsb_no_cl': [],
        'recall_base': [],
        'recall_base_cl': [],
    }

    best_recall_tsb = 0.0
    best_recall_tsb_no_cl = 0.0
    best_recall_base = 0.0
    best_recall_base_cl = 0.0
    
    # Learning Rate Schedulers
    scheduler_tsb = torch.optim.lr_scheduler.StepLR(opt_tsb, step_size=10, gamma=0.9)
    scheduler_tsb_no_cl = torch.optim.lr_scheduler.StepLR(opt_tsb_no_cl, step_size=10, gamma=0.9)
    scheduler_base = torch.optim.lr_scheduler.StepLR(opt_base, step_size=10, gamma=0.9)
    scheduler_base_cl = torch.optim.lr_scheduler.StepLR(opt_base_cl, step_size=10, gamma=0.9)

    for epoch in range(EPOCHS):
        # Shuffle
        perm = np.random.permutation(len(train_data))
        users_np = users_np[perm]
        items_np = items_np[perm]
        
        model_tsb.train()
        model_tsb_no_cl.train()
        model_base.train()
        model_base_cl.train()
        
        # Dynamic SSL Weight
        current_ssl_weight = 0.1 * (0.95 ** epoch)
        
        for i in range(num_batches):
            start_idx = i * BATCH_SIZE
            end_idx = min((i + 1) * BATCH_SIZE, len(train_data))
            
            batch_users = torch.LongTensor(users_np[start_idx:end_idx]).to(device)
            batch_pos = torch.LongTensor(items_np[start_idx:end_idx]).to(device)
            batch_neg = torch.randint(0, utils.num_items, (len(batch_users),)).to(device)
            
            # --- 1. Train TSB-CL (Biclique + CL) ---
            u_g, u_l, u_final, i_final = model_tsb(adj_matrix, (H_v, H_u))
            u_emb = u_final[batch_users]
            pos_emb = i_final[batch_pos]
            neg_emb = i_final[batch_neg]
            pos_scores = torch.sum(u_emb * pos_emb, dim=1)
            neg_scores = torch.sum(u_emb * neg_emb, dim=1)
            loss_bpr = -torch.mean(torch.log(torch.sigmoid(pos_scores - neg_scores) + 1e-8))
            
            u_g_norm = torch.nn.functional.normalize(u_g[batch_users], dim=1)
            u_l_norm = torch.nn.functional.normalize(u_l[batch_users], dim=1)
            pos_sim = torch.sum(u_g_norm * u_l_norm, dim=1)
            loss_cl = -torch.mean(torch.log(torch.sigmoid(pos_sim / 0.2) + 1e-8))
            
            loss_tsb = loss_bpr + current_ssl_weight * loss_cl
            opt_tsb.zero_grad()
            loss_tsb.backward()
            opt_tsb.step()
            
            # --- 2. Train TSB No CL (Biclique Only) ---
            u_g, u_l, u_final, i_final = model_tsb_no_cl(adj_matrix, (H_v, H_u))
            u_emb = u_final[batch_users]
            pos_emb = i_final[batch_pos]
            neg_emb = i_final[batch_neg]
            pos_scores = torch.sum(u_emb * pos_emb, dim=1)
            neg_scores = torch.sum(u_emb * neg_emb, dim=1)
            loss_tsb_no_cl = -torch.mean(torch.log(torch.sigmoid(pos_scores - neg_scores) + 1e-8))
            
            opt_tsb_no_cl.zero_grad()
            loss_tsb_no_cl.backward()
            opt_tsb_no_cl.step()
            
            # --- 3. Train Baseline (LightGCN) ---
            dummy_Hv = torch.sparse_coo_tensor(size=(1, utils.num_items)).to(device)
            dummy_Hu = torch.sparse_coo_tensor(size=(utils.num_users, 1)).to(device)
            u_g_b, _, u_final_b, i_final_b = model_base(adj_matrix, (dummy_Hv, dummy_Hu))
            u_emb_b = u_final_b[batch_users]
            pos_emb_b = i_final_b[batch_pos]
            neg_emb_b = i_final_b[batch_neg]
            pos_scores_b = torch.sum(u_emb_b * pos_emb_b, dim=1)
            neg_scores_b = torch.sum(u_emb_b * neg_emb_b, dim=1)
            loss_base = -torch.mean(torch.log(torch.sigmoid(pos_scores_b - neg_scores_b) + 1e-8))
            
            opt_base.zero_grad()
            loss_base.backward()
            opt_base.step()

            # --- 4. Train Baseline + CL (LightGCN + CL) ---
            # Simulate CL by adding noise to u_g to create a second view
            u_g_b_cl, _, u_final_b_cl, i_final_b_cl = model_base_cl(adj_matrix, (dummy_Hv, dummy_Hu))
            
            # View 1: Original
            u_view1 = u_g_b_cl[batch_users]
            # View 2: Noisy
            noise = torch.rand_like(u_view1) * 0.1 # Small noise
            u_view2 = u_view1 + noise
            
            u_emb_b_cl = u_final_b_cl[batch_users]
            pos_emb_b_cl = i_final_b_cl[batch_pos]
            neg_emb_b_cl = i_final_b_cl[batch_neg]
            
            pos_scores_b_cl = torch.sum(u_emb_b_cl * pos_emb_b_cl, dim=1)
            neg_scores_b_cl = torch.sum(u_emb_b_cl * neg_emb_b_cl, dim=1)
            loss_bpr_cl = -torch.mean(torch.log(torch.sigmoid(pos_scores_b_cl - neg_scores_b_cl) + 1e-8))
            
            # CL Loss between View 1 and View 2
            u_view1_norm = torch.nn.functional.normalize(u_view1, dim=1)
            u_view2_norm = torch.nn.functional.normalize(u_view2, dim=1)
            pos_sim_cl = torch.sum(u_view1_norm * u_view2_norm, dim=1)
            loss_cl_only = -torch.mean(torch.log(torch.sigmoid(pos_sim_cl / 0.2) + 1e-8))
            
            loss_base_cl = loss_bpr_cl + current_ssl_weight * loss_cl_only
            
            opt_base_cl.zero_grad()
            loss_base_cl.backward()
            opt_base_cl.step()
        
        # Step Schedulers
        scheduler_tsb.step()
        scheduler_tsb_no_cl.step()
        scheduler_base.step()
        scheduler_base_cl.step()
            
        # Record Loss
        history['epoch'].append(epoch + 1)
        
        # Evaluate every epoch
        r_tsb, _ = evaluate(model_tsb, True)
        r_tsb_no_cl, _ = evaluate(model_tsb_no_cl, True)
        r_base, _ = evaluate(model_base, False)
        r_base_cl, _ = evaluate(model_base_cl, False)
        
        history['recall_tsb'].append(r_tsb)
        history['recall_tsb_no_cl'].append(r_tsb_no_cl)
        history['recall_base'].append(r_base)
        history['recall_base_cl'].append(r_base_cl)
        
        # Track Best
        if r_tsb > best_recall_tsb:
            best_recall_tsb = r_tsb
            torch.save(model_tsb.state_dict(), os.path.join(MODEL_DIR, 'tsb_cl_best.pth'))
            
        if r_tsb_no_cl > best_recall_tsb_no_cl:
            best_recall_tsb_no_cl = r_tsb_no_cl
            torch.save(model_tsb_no_cl.state_dict(), os.path.join(MODEL_DIR, 'tsb_no_cl_best.pth'))
            
        if r_base > best_recall_base:
            best_recall_base = r_base
            torch.save(model_base.state_dict(), os.path.join(MODEL_DIR, 'base_best.pth'))
            
        if r_base_cl > best_recall_base_cl:
            best_recall_base_cl = r_base_cl
            torch.save(model_base_cl.state_dict(), os.path.join(MODEL_DIR, 'base_cl_best.pth'))
        
        print(f"Ep {epoch+1} | TSB-CL: {r_tsb:.4f} | TSB: {r_tsb_no_cl:.4f} | Base: {r_base:.4f} | Base+CL: {r_base_cl:.4f}")

    print("\\n=== Final Summary (Best Recall) ===")
    print(f"TSB-CL:      {best_recall_tsb:.4f}")
    print(f"TSB (No CL): {best_recall_tsb_no_cl:.4f}")
    print(f"Baseline:    {best_recall_base:.4f}")
    print(f"Base + CL:   {best_recall_base_cl:.4f}")
    
    # Save Final Models
    torch.save(model_tsb.state_dict(), os.path.join(MODEL_DIR, 'tsb_cl_final.pth'))
    torch.save(model_tsb_no_cl.state_dict(), os.path.join(MODEL_DIR, 'tsb_no_cl_final.pth'))
    torch.save(model_base.state_dict(), os.path.join(MODEL_DIR, 'base_final.pth'))
    torch.save(model_base_cl.state_dict(), os.path.join(MODEL_DIR, 'base_cl_final.pth'))
    
    # --- Plotting ---
    print("Generating Plots...")
    plt.figure(figsize=(10, 6))
    
    plt.plot(history['epoch'], history['recall_tsb'], label='TSB-CL (Biclique+CL)', marker='o')
    plt.plot(history['epoch'], history['recall_tsb_no_cl'], label='TSB (Biclique Only)', marker='s')
    plt.plot(history['epoch'], history['recall_base'], label='LightGCN (Baseline)', marker='x')
    plt.plot(history['epoch'], history['recall_base_cl'], label='LightGCN + CL', marker='^')
    
    plt.xlabel('Epochs')
    plt.ylabel('Recall@20')
    plt.title('Performance Comparison (4 Models)')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('training_results.png')
    plt.show()
    print("Plots saved to training_results.png")

if __name__ == "__main__":
    run_quick_proof()
"""
os.makedirs('TSB_CL_Project', exist_ok=True)
with open('TSB_CL_Project/quick_proof_yelp.py', 'w', encoding='utf-8') as f:
    f.write(code)

# [6] Run Quick Proof (Static TSB-CL vs LightGCN)
!python TSB_CL_Project/quick_proof_yelp.py
